# 📊 Diploma Charts Generator

**Мета**: Генерація всіх графіків для дипломної роботи в єдиному науковому стилі.

## Структура графіків:
1. **Signal (CNO)**: Розподіл, boxplot, timeline
2. **Satellites**: Кількість, ефективність
3. **Geometry (DOP)**: Кореляція, розподіл
4. **Accuracy**: Histogram, Physics proof

## Вихід:
- Всі графіки зберігаються в `diploma_charts/` з роздільністю 300 DPI

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc

# Налаштування
OUTPUT_DIR = '../diploma_charts'
DATA_PATH = '../data/processed/all_data_compressed.parquet'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Науковий стиль
plt.style.use('seaborn-v0_8-whitegrid')
colors = ["#2c3e50", "#e74c3c", "#3498db", "#27ae60", "#f1c40f"]
sns.set_palette(sns.color_palette(colors))

print("✅ Налаштування завершено")
print(f"📁 Output directory: {os.path.abspath(OUTPUT_DIR)}")

## 1. Завантаження даних (оптимізоване)

In [ ]:
# Тільки потрібні колонки для економії RAM
REQUIRED_COLUMNS = [
    'timestamp',
    'cnoMean',
    'numSV',
    'numSatsTracked',
    'hAcc',
    'pDOP', 'vDOP', 'hDOP'
]

print("⏳ Завантаження даних...")

try:
    df = pd.read_parquet(DATA_PATH, columns=REQUIRED_COLUMNS, engine='pyarrow')
    
    # Downcasting для економії пам'яті
    fcols = df.select_dtypes('float').columns
    df[fcols] = df[fcols].astype('float32')
    icols = df.select_dtypes('integer').columns
    df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')
    
    # Конвертація часу
    if not np.issubdtype(df['timestamp'].dtype, np.datetime64):
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    print(f"✅ Завантажено: {len(df):,} записів")
    print(f"🧠 Використання пам'яті: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
except Exception as e:
    print(f"❌ Помилка: {e}")

# Sample для важких графіків
df_sample = df.sample(n=min(100000, len(df)), random_state=42)
print(f"📊 Sample size: {len(df_sample):,} (для scatter plots)")

## 2. ГРУПА 1: Signal (CNO)

In [ ]:
print("📊 Генерація Signal Charts...")

# 1. CNO Distribution (Histogram + KDE)
plt.figure(figsize=(10, 6))
sns.histplot(df['cnoMean'], bins=60, kde=True, color=colors[0], stat="percent")
plt.axvline(x=25, color=colors[1], linestyle='--', linewidth=2, label='Jamming Threshold (<25 dBHz)')
plt.title('Distribution of Signal Strength (CNO)', fontsize=14, fontweight='bold')
plt.xlabel('Carrier-to-Noise Density (dBHz)', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.legend()
plt.savefig(f'{OUTPUT_DIR}/01_cno_distribution.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# 2. CNO Boxplot (Stability)
plt.figure(figsize=(8, 5))
sns.boxplot(x=df_sample['cnoMean'], color=colors[2])
plt.title('Signal Stability Analysis (Boxplot)', fontsize=14, fontweight='bold')
plt.xlabel('CNO (dBHz)', fontsize=12)
plt.savefig(f'{OUTPUT_DIR}/02_cno_boxplot.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# 3. CNO Timeline (Fragment with drop)
subset = df.iloc[5000:7000]
plt.figure(figsize=(12, 5))
plt.plot(subset['timestamp'], subset['cnoMean'], color=colors[0], linewidth=1)
plt.title('Signal Drop Detection (Timeline Fragment)', fontsize=14, fontweight='bold')
plt.xlabel('Time', fontsize=12)
plt.ylabel('CNO (dBHz)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.savefig(f'{OUTPUT_DIR}/03_cno_timeline.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

## 3. ГРУПА 2: Satellites

In [ ]:
print("🛰️ Генерація Satellite Charts...")

# 4. Satellite Count Distribution
plt.figure(figsize=(10, 6))
sns.countplot(x=df_sample['numSV'], color=colors[2])
plt.title('Visible Satellites Count Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Number of Visible Satellites', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.savefig(f'{OUTPUT_DIR}/04_satellites_count.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# 5. Satellite Efficiency (Tracked vs Visible)
if 'numSatsTracked' in df.columns:
    plt.figure(figsize=(8, 8))
    plt.hist2d(df_sample['numSV'], df_sample['numSatsTracked'], bins=30, cmap='Blues')
    plt.colorbar(label='Frequency')
    plt.plot([0, 40], [0, 40], 'r--', linewidth=2, label='Ideal 1:1')
    plt.title('Efficiency: Visible vs Tracked Satellites', fontsize=14, fontweight='bold')
    plt.xlabel('Visible Satellites (numSV)', fontsize=12)
    plt.ylabel('Tracked Satellites (Used)', fontsize=12)
    plt.legend()
    plt.savefig(f'{OUTPUT_DIR}/05_sat_efficiency.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

## 4. ГРУПА 3: Geometry (DOP)

In [ ]:
print("📐 Генерація Geometry Charts...")

# 6. DOP Correlation Heatmap
dop_cols = [c for c in df.columns if 'dop' in c.lower()]
if dop_cols:
    plt.figure(figsize=(8, 6))
    sns.heatmap(df[dop_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f", 
                square=True, linewidths=0.5)
    plt.title('Correlation Matrix: Geometric Factors (DOP)', fontsize=14, fontweight='bold')
    plt.savefig(f'{OUTPUT_DIR}/06_dop_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
# 7. PDOP Distribution (Violin)
if 'pDOP' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.violinplot(x=df_sample['pDOP'], color=colors[3])
    plt.title('PDOP Density Distribution', fontsize=14, fontweight='bold')
    plt.xlabel('Position Dilution of Precision', fontsize=12)
    plt.savefig(f'{OUTPUT_DIR}/07_pdop_violin.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

## 5. ГРУПА 4: Accuracy (Target)

In [ ]:
print("🎯 Генерація Accuracy Charts...")

# 8. hAcc Histogram (Log Scale)
plt.figure(figsize=(10, 6))
sns.histplot(df['hAcc'], bins=100, log_scale=True, color=colors[1])
plt.axvline(x=5000, color='green', linestyle='--', linewidth=2, label='Normal (<5m)')
plt.axvline(x=50000, color='red', linestyle='--', linewidth=2, label='Critical (>50m)')
plt.title('Horizontal Accuracy Error Distribution (Log Scale)', fontsize=14, fontweight='bold')
plt.xlabel('Error (mm)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.legend()
plt.savefig(f'{OUTPUT_DIR}/08_hacc_log_hist.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# 9. PHYSICS PROOF: CNO vs hAcc ("Hockey Stick")
plt.figure(figsize=(10, 6))
plt.scatter(df_sample['cnoMean'], df_sample['hAcc'], alpha=0.2, s=15, color='#8e44ad')
plt.yscale('log')
plt.title('PHYSICS PROOF: Signal Strength vs Accuracy Error', fontsize=14, fontweight='bold')
plt.xlabel('Signal Strength (CNO, dBHz)', fontsize=12)
plt.ylabel('Accuracy Error (mm) [Log Scale]', fontsize=12)
plt.grid(True, which='both', linestyle='--', alpha=0.5)
plt.savefig(f'{OUTPUT_DIR}/09_physics_proof.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# 10. Global Pairplot
print("   Генерація pairplot (може зайняти час)...")
cols_to_plot = ['cnoMean', 'numSV', 'hAcc']
if 'pDOP' in df.columns:
    cols_to_plot.append('pDOP')

g = sns.pairplot(df_sample[cols_to_plot], diag_kind='kde', 
                 plot_kws={'alpha': 0.1, 's': 5})
g.fig.suptitle('Global Feature Relationships', y=1.02, fontsize=14, fontweight='bold')
plt.savefig(f'{OUTPUT_DIR}/10_global_pairplot.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

## 📋 Підсумок

In [ ]:
# Список згенерованих файлів
generated_files = sorted(os.listdir(OUTPUT_DIR))

print("="*60)
print("✅ DIPLOMA CHARTS GENERATION COMPLETE!")
print("="*60)
print(f"\n📁 Output directory: {os.path.abspath(OUTPUT_DIR)}")
print(f"\n📊 Generated {len(generated_files)} charts:")
for f in generated_files:
    print(f"   • {f}")

# Очищення пам'яті
gc.collect()
print("\n🧹 Memory cleaned.")